## Подготавливаем данные

В качестве исходного материала для чат-бота будем использовать ролики с YouTube. Для того, чтобы с ними работать, установим библиотеку [`PyTube`](https://pytube.io/).

In [1]:
%pip install pytube

Defaulting to user installation because normal site-packages is not writeable


Возможно, прежде чем выполнять следующий код, вам потребуется перезапустить Kernel.

### Скачиваем звуковые дорожки к видео

Посмотрим, как можно обратиться к ролику на YouTube по ссылке:

In [30]:
from pytube import YouTube
yt = YouTube("https://www.youtube.com/watch?v=7mWPlXvPG7A")
yt.title,yt.streams

('Yandex Scale 2022. Главный доклад.',
 [<Stream: itag="17" mime_type="video/3gpp" res="144p" fps="6fps" vcodec="mp4v.20.3" acodec="mp4a.40.2" progressive="True" type="video">, <Stream: itag="18" mime_type="video/mp4" res="360p" fps="25fps" vcodec="avc1.42001E" acodec="mp4a.40.2" progressive="True" type="video">, <Stream: itag="22" mime_type="video/mp4" res="720p" fps="25fps" vcodec="avc1.64001F" acodec="mp4a.40.2" progressive="True" type="video">, <Stream: itag="137" mime_type="video/mp4" res="1080p" fps="25fps" vcodec="avc1.640028" progressive="False" type="video">, <Stream: itag="248" mime_type="video/webm" res="1080p" fps="25fps" vcodec="vp9" progressive="False" type="video">, <Stream: itag="136" mime_type="video/mp4" res="720p" fps="25fps" vcodec="avc1.4d401f" progressive="False" type="video">, <Stream: itag="247" mime_type="video/webm" res="720p" fps="25fps" vcodec="vp9" progressive="False" type="video">, <Stream: itag="135" mime_type="video/mp4" res="480p" fps="25fps" vcodec="av

Мы видим, что с каждым видео связано несколько потоков, включая звуковые. Мы можем отфильтровать нужные нам потоки, взять первый из них, и сохранить на диск:

In [3]:
yt.streams.filter(mime_type="audio/webm").first()#.download(output_path="../audio",filename="1.opus")

'/home/jupyter/work/resources/VideoQABot/../audio/1.opus'

Возьмём теперь коллекцию видео, и сохраним все звуковые дорожки от них:

In [31]:
videos = ['https://www.youtube.com/watch?v=2T1hRvnIu1U',
          'https://www.youtube.com/watch?v=9O8eEmlSiBw',
          'https://www.youtube.com/watch?v=mPvLf-TqS74']

for i,url in enumerate(videos):
    yt = YouTube(url)
    print(f"Downloading {yt.title}")
    yt.streams.filter(mime_type="audio/webm").first().download(output_path="../audio",filename=f"{i}.opus")

Для того, чтобы SpeechKit мог распознать речь в файле, файл должен быть в определённом формате и битрейте. Поэтому откроем все наши звуковые дорожки, и сделаем им ресамплинг к частоте 8 kHz с помощью библиотеки `librosa`. Этот процесс может занять некоторое время.

> Вы также можете произвести преобразование форматов и битрейта из командной строки с помощью утилиты `ffmpeg`

In [32]:
import glob
import librosa
import soundfile as sf

target_sr = 8000
for fn in glob.glob("../audio/*.opus"):
    print(f"Processing {fn}")
    au,sr = librosa.load(fn,sr=target_sr)
    sf.write(fn.replace('.opus','.ogg'),au,target_sr,format='ogg',subtype='opus')
    

Processing ../audio/1.opus


<ipython-input-5-eb729a1953cc>:8: UserWarning: PySoundFile failed. Trying audioread instead.
  au,sr = librosa.load(fn,sr=target_sr)
/usr/local/lib/python3.10/dist-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


Processing ../audio/0.opus


<ipython-input-5-eb729a1953cc>:8: UserWarning: PySoundFile failed. Trying audioread instead.
  au,sr = librosa.load(fn,sr=target_sr)
/usr/local/lib/python3.10/dist-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


Processing ../audio/2.opus


<ipython-input-5-eb729a1953cc>:8: UserWarning: PySoundFile failed. Trying audioread instead.
  au,sr = librosa.load(fn,sr=target_sr)
/usr/local/lib/python3.10/dist-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


### Распознаём речь с помощью SpeechKit

У SpeechKit есть несколько режимов работы. Для распознавания больших файлов лучше всего подходит [асинхронное распознавание](https://cloud.yandex.ru/docs/speechkit/stt/transcribation). Для асинхронного распознавания необходимо, чтобы исходные файлы лежали в хранилище Yandex S3.

Копируем аудио-файлы в хранилище S3, подключенное к DataSphere, чтобы можно было использовать их для распознавания речи.

In [33]:
!cp ../audio/*.ogg /home/jupyter/mnt/s3/mclass/audio

Для работы со Speech Kit потребуется [создать ключ API](https://cloud.yandex.ru/docs/iam/operations/api-key/create). Далее, чтобы не указывать ключ в программном коде, нужно определить ключи как **секреты** в DataSphere. После того, как вы это сделаете, ключи станут доступны просто как переменные окружения.

Весь процесс работы [описан в документации](https://cloud.yandex.ru/docs/speechkit/stt/api/transcribation-api).

In [35]:
import os

api_id = os.environ['api_id']
api_key = os.environ['api_key']

Для запуска распознавания конкретного файла необходимо сделать соответствующий POST-запрос.

In [54]:
import requests

def submit_for_sr(audio_file):
    j = {
        "config": {
            "specification": {
                "languageCode": "ru-RU",
            }
        },
        "audio": {
            "uri": audio_file
        }
    }
    res = requests.post("https://transcribe.api.cloud.yandex.net/speech/stt/v2/longRunningRecognize",
                    json = j,
                    headers = { "Authorization" : f"Api-Key {api_key}" })
    return res.json()['id']

Инициируем распознавание первого файла. Если всё хорошо, в ответ мы получим `id`, который можно использовать для проверки результата операции.

In [55]:
id = submit_for_sr('https://storage.yandexcloud.net/mclass/audio/0.ogg')
id

'e03vn8vnh3fugsbq5629'

Следующую ячейку можно запускать несколько раз, пока вы не получите результат распознавания.

In [59]:
requests.get(f"https://operation.api.cloud.yandex.net/operations/{id}",headers = { "Authorization" : f"Api-Key {api_key}" }).json()

Теперь запустим процесс распознавания для всех наших файлов, и затем будем ждать, пока все результаты не будут получены:

In [60]:
d = {}
for fn in glob.glob('/home/jupyter/mnt/s3/mclass/audio/*.ogg'):
    ext_name = fn.replace('/home/jupyter/mnt/s3/','https://storage.yandexcloud.net/')
    id = submit_for_sr(ext_name)
    print(f"Submitted {fn} -> {id}")
    d[id] = fn    

Submitted /home/jupyter/mnt/s3/mclass/audio/0.ogg -> e03igm0in62f6r8mrmoq
Submitted /home/jupyter/mnt/s3/mclass/audio/1.ogg -> e03741lh2iu7ifg5nm2t
Submitted /home/jupyter/mnt/s3/mclass/audio/2.ogg -> e03mjcnj77hvcdiicv36


In [62]:
import time 

def check_ready(id):
    res = requests.get(f"https://operation.api.cloud.yandex.net/operations/{id}",headers = { "Authorization" : f"Api-Key {api_key}" })
    res = res.json()
    if res['done']:
        return res['response']
    else:
        return None

txt = {}
while True:
    for k,v in d.items():
        if v in txt.keys():
            continue
        res = check_ready(k)
        if res is None:
            print(f"{k} -> waiting")
        else:
            print(f"{k} -> ready")
            txt[v] = ' '.join([x['alternatives'][0]['text'] for x in res['chunks']])
    if len(txt.keys())==len(d.keys()):
        break
    time.sleep(10)

e03igm0in62f6r8mrmoq -> waiting
e03741lh2iu7ifg5nm2t -> waiting
e03mjcnj77hvcdiicv36 -> waiting
e03igm0in62f6r8mrmoq -> waiting
e03741lh2iu7ifg5nm2t -> waiting
e03mjcnj77hvcdiicv36 -> waiting
e03igm0in62f6r8mrmoq -> waiting
e03741lh2iu7ifg5nm2t -> waiting
e03mjcnj77hvcdiicv36 -> waiting
e03igm0in62f6r8mrmoq -> waiting
e03741lh2iu7ifg5nm2t -> waiting
e03mjcnj77hvcdiicv36 -> waiting
e03igm0in62f6r8mrmoq -> waiting
e03741lh2iu7ifg5nm2t -> waiting
e03mjcnj77hvcdiicv36 -> waiting
e03igm0in62f6r8mrmoq -> waiting
e03741lh2iu7ifg5nm2t -> waiting
e03mjcnj77hvcdiicv36 -> waiting
e03igm0in62f6r8mrmoq -> waiting
e03741lh2iu7ifg5nm2t -> waiting
e03mjcnj77hvcdiicv36 -> waiting
e03igm0in62f6r8mrmoq -> waiting
e03741lh2iu7ifg5nm2t -> waiting
e03mjcnj77hvcdiicv36 -> waiting
e03igm0in62f6r8mrmoq -> waiting
e03741lh2iu7ifg5nm2t -> waiting
e03mjcnj77hvcdiicv36 -> waiting
e03igm0in62f6r8mrmoq -> waiting
e03741lh2iu7ifg5nm2t -> waiting
e03mjcnj77hvcdiicv36 -> waiting
e03igm0in62f6r8mrmoq -> waiting
e03741lh

Теперь сохраним результаты распознавания в текстовые файлы:

In [63]:
for k,v in txt.items():
    with open(k.replace('.ogg','.txt').replace('/audio/','/text/'),'w',encoding='utf-8') as f:
        f.write(v)

Мы получили коллекцию текстовых документов, в которых теперь можно организовать полнотекстовый поиск и построить вопрос-ответного бота. Переходим к следующему этапу работы: `LangChainQA.ipynb`